# Adding context to your prompts - Retrieval Augmented Generation (RAG)

In [1]:
#r "nuget: Azure.AI.OpenAI, 1.0.0-beta.16"
#r "nuget: Microsoft.DotNet.Interactive, 1.0.0-beta.24229.4"
#r "nuget: System.Numerics.Tensors, 8.0.0"

Installed Packages Azure.AI.OpenAI, 1.0.0-beta.16 Microsoft.DotNet.Interactive, 1.0.0-beta.24229.4 System.Numerics.Tensors, 8.0.0

In [2]:
using System.Collections.Generic;
using System.IO;
using System.Linq;
using System.Text.Json;
using Azure.AI.OpenAI;
using Microsoft.DotNet.Interactive;
using System.Numerics.Tensors;

## Get OpenAI API Key

In [3]:
var key = Environment.GetEnvironmentVariable("OAI_KEY");

## Configure OpenAI client

In [4]:
var client = new OpenAIClient(key);

## Define system prompt

In [5]:
var systemPrompt = 
    """
    You are an AI Assistant that helps user answer questions about their data. 
    
    You have access to articles to help you answer questions. 
    
    Use only the contents from these documents to answer questions. 
    """;

## Define Data Objects

In [6]:
public class Data
{
    public string Text { get; set; }
    public float[] Embedding { get; set; }
}

## Utility Functions

In [7]:
string FormatMessage(ChatRequestMessage m)
{
    if (m.Role == ChatRole.System)
        return $"System: {((ChatRequestSystemMessage)m).Content}";
    else if (m.Role == ChatRole.User)
        return $"User: {((ChatRequestUserMessage)m).Content}";
    else if (m.Role == ChatRole.Assistant)
        return $"Assistant: {((ChatRequestAssistantMessage)m).Content}";
    else
        return m.ToString();
}

string RenderUserPrompt(IEnumerable<ChatRequestMessage> history, string[] documents, string query)
{
    var chatMessages =
        history
            .Select(FormatMessage)
            .Select(x => $"{x}\n")
            .ToArray();

    var sources = string.Join('\n', documents);

    var template =
        $"""
        ## Chat History

        {String.Join('\n', chatMessages)}

        ## Sources

        {sources}

        ## Query
        {query}
        """;

    return template;
}

IEnumerable<Data> GetTopKDocuments(IEnumerable<Data> sources, float[] userQueryEmbedding, int k)
{
    return sources.Select(document =>
        {
            var similarity = TensorPrimitives.CosineSimilarity(document.Embedding, userQueryEmbedding);
            return (similarity, document);
        })
        .OrderByDescending(x => x.similarity)
        .Take(k)
        .Select(x => x.document)
        .ToArray();
}

## Load data

In [8]:
var data = File.ReadAllText("Data/03-embedding.json");
var sources = JsonSerializer.Deserialize<Data[]>(data);

## Display first data entry

In [9]:
sources.First()

Submission#6+Data Text Cristiano Ronaldo dos Santos Aveiro GOIH ComM (Portuguese pronunciation: [kɾiʃˈtjɐnu ʁɔˈnaldu]; born 5 February 1985) is a Portuguese professional footballer who plays as a forward for and captains both Saudi Pro League club Al Nassr and the Portugal national team. Widely regarded as one of the greatest players of all time, Ronaldo has won five Ballon d'Or awards,[note 3] a record three UEFA Men's Player of the Year Awards, and four European Golden Shoes, the most by a European player. He has won 33 trophies in his career, including seven league titles, five UEFA Champions Leagues, the UEFA European Championship and the UEFA Nations League. Ronaldo holds the records for most appearances (183), goals (140) and assists (42) in the Champions League, goals in the European Championship (14), international goals (128) and international appearances (206). He is one of the few players to have made over 1,200 professional career appearances, the most by an outfield player, and has scored over 890 official senior career goals for club and country, making him the top goalscorer of all time.

Ronaldo began his senior career with Sporting CP, before signing with Manchester United in 2003, winning the FA Cup in his first season. He would also go on to win three consecutive Premier League titles, the Champions League and the FIFA Club World Cup; at age 23, he won his first Ballon d'Or. Ronaldo was the subject of the then-most expensive association football transfer when he signed for Real Madrid in 2009 in a transfer worth €94 million (£80 million). He became a key contributor and formed an attacking trio with Karim Benzema and Gareth Bale which was integral to the team winning four Champions Leagues from 2014 to 2018, including La Décima. During this period, he won back-to-back Ballons d'Or in 2013 and 2014, and again in 2016 and 2017, and was runner-up three times behind Lionel Messi, his perceived career rival. He also became the club's all-time top goalscorer and the all-time top scorer in the Champions League, and finished as the competition's top scorer for six consecutive seasons between 2012 and 2018. With Real, Ronaldo won four Champions Leagues, two La Liga titles, two Copas del Rey, two UEFA Super Cups and three Club World Cups. In 2018, he signed for Juventus in a transfer worth an initial €100 million (£88 million), the most expensive transfer for an Italian club and for a player over 30 years old. He won two Serie A titles, two Supercoppa Italiana trophies and a Coppa Italia, became the inaugural Serie A Most Valuable Player and became the first footballer to finish as top scorer in the English, Spanish and Italian leagues. He returned to Manchester United in 2021, finishing his only full season as the club's top scorer, before his contract was terminated in 2022. In 2023, he signed for Al Nassr.

Ronaldo made his international debut for Portugal in 2003 at the age of 18 and has since earned more than 200 caps, making him both the country and history's most-capped player of all time, recognised by the Guinness World Records.[10] With more than 100 goals at international level, he is also the sports all-time top goalscorer. Ronaldo has played in and scored at eleven major tournaments; he scored his first international goal at Euro 2004, where he helped Portugal reach the final. He assumed captaincy of the national team in July 2008. In 2015, Ronaldo was named the best Portuguese player of all time by the Portuguese Football Federation. The following year, he led Portugal to their first major tournament title at Euro 2016, and received the Silver Boot as the second-highest goalscorer of the tournament. This achievement would see him receive his fourth Ballon d'Or. He also led them to victory in the inaugural UEFA Nations League in 2019, receiving the top scorer award in the finals, and later received the Golden Boot as top scorer of Euro 2020.

One of the world's most marketable and famous athletes, R

## Configure Chat Completion options


In [10]:
var options = new ChatCompletionsOptions
{
    MaxTokens = 400, // https://platform.openai.com/tokenizer
    Temperature = 1f, // Accuracy 0f highest, 1f creativity mode
    FrequencyPenalty = 0.0f,
    PresencePenalty = 0.0f,
    NucleusSamplingFactor = 0.95f,
    DeploymentName = "gpt-4o" // Models: https://platform.openai.com/docs/models
};

## Initialize chat history

In [11]:
var messages = new List<ChatRequestMessage>();
var systemMessage = new ChatRequestSystemMessage(systemPrompt);
messages.Add(systemMessage);
options.Messages.Add(systemMessage);

## Start chat

In [12]:
var chatting = true;

In [14]:
Console.WriteLine($"{FormatMessage(systemMessage)}");

while(chatting)
{
    //Get User input
    var userInput = await Kernel.GetInputAsync("Please enter your prompt. Press 'q' to quit.");
    if(userInput.ToLowerInvariant() == "q")
    {
        chatting = false;
        break;    
    }
    
    // Query relevant documents
    var embeddingOptions = new EmbeddingsOptions
    {
        DeploymentName = "text-embedding-ada-002",
        Input={userInput}
    };

    var embeddingResponse = await client.GetEmbeddingsAsync(embeddingOptions);
    var inputEmbedding = embeddingResponse.Value.Data[0].Embedding.ToArray();
    var docs = 
        GetTopKDocuments(sources, inputEmbedding, 3)
            .Select(doc => doc.Text)
            .ToArray();

    // Generate User Prompt
    messages.Add(new ChatRequestUserMessage(userInput));
    var query = RenderUserPrompt(messages, docs, userInput);
    var userMessage = new ChatRequestUserMessage(query);
    options.Messages.Add(userMessage);
    Console.WriteLine(query);
    
    // Generate response
    var assistantResponse = await client.GetChatCompletionsAsync(options);
    var response = assistantResponse.Value.Choices[0].Message.Content;
    var assistantMessage = new ChatRequestAssistantMessage(response);
    Console.WriteLine($"\n## Response\n{FormatMessage(assistantMessage)}\n");
    messages.Add(assistantMessage);
    options.Messages.Add(assistantMessage);
    Console.WriteLine("-----");
    Console.WriteLine("");
};

messages.Clear();
options.Messages.Clear();

System: You are an AI Assistant that helps user answer questions about their data. 

You have access to articles to help you answer questions. 

Use only the contents from these documents to answer questions. 
## Chat History

System: You are an AI Assistant that helps user answer questions about their data. 

You have access to articles to help you answer questions. 

Use only the contents from these documents to answer questions. 

User: How many goals did Cristiano Ronaldo score in 2014?


## Sources

Cristiano Ronaldo dos Santos Aveiro GOIH ComM (Portuguese pronunciation: [kɾiʃˈtjɐnu ʁɔˈnaldu]; born 5 February 1985) is a Portuguese professional footballer who plays as a forward for and captains both Saudi Pro League club Al Nassr and the Portugal national team. Widely regarded as one of the greatest players of all time, Ronaldo has won five Ballon d'Or awards,[note 3] a record three UEFA Men's Player of the Year Awards, and four European Golden Shoes, the most by a European player.